In [16]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

### Data Processing and Feature Engineering

In [2]:
#### Loading all data into dataframes
train_df = pd.read_csv('../data/train.csv')
holidays_df = pd.read_csv('../data/holidays_events.csv')
oil_df = pd.read_csv('../data/oil.csv')
stores_df = pd.read_csv('../data/stores.csv')
transactions_df = pd.read_csv('../data/transactions.csv')

## Will be used in the future
#sample_df = pd.read_csv('../data/sample_submission.csv')
#test_df = pd.read_csv('../data/test.csv')

In [ ]:
def holidays_feat(df):

    """
    This function removes transferred holidays and organize the holidays dataframe
    as 3 boolean approach to merge with train dataframe
    End result:
    is_nat_hol -> If it is national holiday
    is_reg_hol -> If it is regional holiday
    is_loc_hol -> If it is local holiday

    Input: Holidays dataframe | Columns: ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']

    Output: Holidays dictionary with 3 objects:
        Key: 'Local'    | Value: Dataframe with Columns: ['date', 'locale_name', 'is_local_hld']
        Key: 'Regional' | Value: Dataframe with Columns: ['date', 'locale_name', 'is_regional_hld']
        Key: 'National' | Value: Dataframe with Columns: ['date', 'locale_name', 'is_national_hld']
    """
    
    #### Creates a copy of the original dataframe to manipulate the data
    temp_df = df[df['transferred'] == False].copy()
    temp_df['date'] = pd.to_datetime(temp_df['date'])

    #### Initiates an empty dictionary and loops through each 'locale' to create each dataframe
    hol_dict = {}
    for i in temp_df['locale'].unique():
        temp_df2 = (temp_df[temp_df['locale'] == i][['date','locale_name']]
                    .drop_duplicates())
        temp_df2[f'is_{i}_hld'.lower()] = True
        hol_dict.update({i: temp_df2})
    print(f'The dictionary has been created')
    return hol_dict

def oil_feat(df):
    """
    This function rework the oil dataframe to include include weekend missing data with backfill

    Input: Oil dataframe | Columns: ['date', 'dcoilwtico']

    Output: Oil dataframe with backfill | Columns: ['date', 'dcoilwtico']

    """

    #### Create a dataframe with the first and last date in df and set the freq to daily
    temp_df = df.copy()
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    totaldays = pd.DataFrame({'date': pd.date_range(start = temp_df['date'].min(), end = temp_df['date'].max())})
    totaldays['date'].freq = 'd'

    #### Merge the two dataframes and do a backward fill to get the correct values of the day
    #### then we transform the column to datetime format
    oil_df_final = totaldays.merge(temp_df, how = 'left', on = 'date').bfill()
    #oil_df_final['date'] = pd.to_datetime(oil_df_final['date'])
    return oil_df_final

In [4]:
holidays = holidays_feat(holidays_df)

The dictionary has been created


In [10]:
oil = oil_feat(oil_df)

In [ ]:
def cluster_feat(df):
    # Group and aggregate sales and promotions
    prd_sales_df = (
        df
        .groupby(['date', 'family'], as_index=False)[['sales']]
        .sum()
    )

    # Extract quarter from date
    prd_sales_df['quarter'] = pd.to_datetime(prd_sales_df['date']).dt.quarter

    # Pivot sales per quarter
    prd_sales_df2 = (
        prd_sales_df
        .pivot_table(index='family', columns='quarter', values='sales', aggfunc='sum')
        .fillna(0)  # Ensure no NaNs in sales
        .round(2)
    )

    # Normalize quarterly sales
    total_sales = prd_sales_df2.sum(axis=1)
    df_prd_norm = (prd_sales_df2.div(total_sales, axis=0)).round(4)

    # Rename columns
    df_prd_norm.columns = [f"Q{col}" for col in df_prd_norm.columns]

    # Create a Scaler and fit the values
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_prd_norm)

    # Fit predict the data
    kmeans = KMeans(n_clusters=4, random_state = 265)
    clusters = kmeans.fit_predict(scaled_data)

    # Reinsert the data on the dataframe
    df_prd_norm['cluster'] = clusters

In [ ]:
# Group and aggregate sales and promotions
prd_sales_df = (
    train_df
    .groupby(['date', 'family'], as_index=False)[['sales']]
    .sum()
)

# Extract quarter from date
prd_sales_df['quarter'] = pd.to_datetime(prd_sales_df['date']).dt.quarter

# Pivot sales per quarter
prd_sales_df2 = (
    prd_sales_df
    .pivot_table(index='family', columns='quarter', values='sales', aggfunc='sum')
    .fillna(0)  # Ensure no NaNs in sales
    .round(2)
)

# Normalize quarterly sales
total_sales = prd_sales_df2.sum(axis=1)
df_prd_norm = (prd_sales_df2.div(total_sales, axis=0)).round(4)

# Rename columns
df_prd_norm.columns = [f"Q{col}" for col in df_prd_norm.columns]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_prd_norm)
kmeans = KMeans(n_clusters=4, random_state = 265)
clusters = kmeans.fit_predict(scaled_data)
df_prd_norm['cluster'] = clusters

In [36]:
test = pd.get_dummies(df_prd_norm['cluster'])

In [37]:
test

,0,1,2,3
family,,,,
AUTOMOTIVE,False,False,False,True
BABY CARE,False,False,False,True
BEAUTY,False,False,False,True
BEVERAGES,False,False,False,True
BOOKS,False,True,False,False
BREAD/BAKERY,False,False,False,True
CELEBRATION,True,False,False,False
CLEANING,False,False,False,True
DAIRY,False,False,False,True


In [24]:
df_prd_norm

,Q1,Q2,Q3,Q4,cluster
family,,,,,
AUTOMOTIVE,0.2697,0.2669,0.2393,0.2241,3
BABY CARE,0.2601,0.2323,0.3204,0.1872,3
BEAUTY,0.2475,0.2549,0.2573,0.2403,3
BEVERAGES,0.2524,0.2545,0.2520,0.2411,3
BOOKS,0.2788,0.0730,0.0084,0.6398,1
BREAD/BAKERY,0.2605,0.2737,0.2508,0.2150,3
CELEBRATION,0.2298,0.2071,0.2787,0.2844,0
CLEANING,0.2659,0.2698,0.2419,0.2224,3
DAIRY,0.2630,0.2755,0.2336,0.2278,3
